In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
import requests
import bs4
from bs4 import BeautifulSoup


import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("./seoul_food.csv", encoding='cp949')
print(df.shape)
df.head()

(114055, 6)


,구,동,사업장명,업종,좌표정보(X),좌표정보(Y)
0,종로구,관철동,상해,분식,198867.702591,451824.281126
1,관악구,신림동,미쿠,일식,193687.348199,442284.164025
2,구로구,구로동,여름엔빙수겨울엔떡국,한식,189814.305919,443046.239859
3,강남구,대치동,카페디퍼,카페,204776.548264,444677.642985
4,동대문구,신설동,메이크데이(MakeDay),카페,202114.228397,452392.093759


### 1. requests과 공공데이터를 이용해서 리뷰주소 크롤링

In [ ]:
id_list = []
cnt = 0
for i in range(len(id_list),df.shape[0]):
    keyword = df['구'][i] +' '+ df['동'][i] +' '+ df['사업장명'][i]
    url = f'https://search.map.kakao.com/mapsearch/map.daum?callback=jQuery18105963491453995977_1649742498249&q={keyword}&msFlag=A&sort=0'
    header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
        'referer': 'https://map.kakao.com/'
    }
    response = requests.get(url, headers=header)
    try:
        confirmid = response.text.split('"confirmid":"')[1].split('","x":')[0]
        review_address = f'https://place.map.kakao.com/{confirmid}#comment'
        id_list.append(review_address)
        cnt += 1
    except:
        id_list.append('error')
        cnt += 1
    if cnt % 1000 == 0:
        print(f'{cnt}개 진행완료')

In [3]:
df.insert(6,'리뷰주소',id_list)
print(df.shape)
df.head()

(114055, 7)


,구,동,사업장명,업종,좌표정보(X),좌표정보(Y),리뷰주소
0,종로구,관철동,상해,분식,198867.702591,451824.281126,https://place.map.kakao.com/13333296#comment
1,관악구,신림동,미쿠,일식,193687.348199,442284.164025,https://place.map.kakao.com/658611073#comment
2,구로구,구로동,여름엔빙수겨울엔떡국,한식,189814.305919,443046.239859,https://place.map.kakao.com/1160052313#comment
3,강남구,대치동,카페디퍼,카페,204776.548264,444677.642985,https://place.map.kakao.com/637651089#comment
4,동대문구,신설동,메이크데이(MakeDay),카페,202114.228397,452392.093759,error


### 2. 카카오맵에 존재하지 않는 음식점 제거

In [4]:
error = len(df[df['리뷰주소'] == 'error'])
print(f'카카오맵에 존재하지 않는 음식점{error}개')

카카오맵에 존재하지 않는 음식점22715개


In [5]:
df = df[df['리뷰주소'] != 'error']
df = df.reset_index(drop=True)
print(df.shape)
df.head()

(91340, 7)


,구,동,사업장명,업종,좌표정보(X),좌표정보(Y),리뷰주소
0,종로구,관철동,상해,분식,198867.702591,451824.281126,https://place.map.kakao.com/13333296#comment
1,관악구,신림동,미쿠,일식,193687.348199,442284.164025,https://place.map.kakao.com/658611073#comment
2,구로구,구로동,여름엔빙수겨울엔떡국,한식,189814.305919,443046.239859,https://place.map.kakao.com/1160052313#comment
3,강남구,대치동,카페디퍼,카페,204776.548264,444677.642985,https://place.map.kakao.com/637651089#comment
4,은평구,응암동,달의 작업실(Lalune),한식,192540.370555,454712.243026,https://place.map.kakao.com/1966713560#comment


### 3. 카카오맵에서 음식점 이름 및 주소 갱신

In [ ]:
gu_list=[]
add_list=[]
title_list=[]
cnt=0
print(f'{cnt}개 진행완료')
for i in range(len(gu_list),df.shape[0]):
    url = df['리뷰주소'][i]
    header = {'user-agent': 'Mozilla/5.0'}
    res = requests.get(url, headers=header)
    soup = BeautifulSoup(res.text, 'html.parser')
    title = soup.head.find('meta',{'property':"og:title"}).get('content')
    address = soup.head.find('meta',{'property':"og:description"}).get('content')
    try:
        gu = address.split()[1]
        add_list.append(address)
        gu_list.append(gu)
        title_list.append(title)
        cnt += 1
    except:
        add_list.append('error')
        gu_list.append('error')
        title_list.append('error')
        cnt += 1
    if cnt % 1000 == 0 :
        print(f'{cnt}개 진행완료')

In [ ]:
df.insert(7,'주소(카카오)',add_list)
df.insert(8,'사업장명(카카오)',title_list)

In [6]:
print(df.shape)
df.head()

(91340, 9)


,구,동,사업장명,업종,좌표정보(X),좌표정보(Y),리뷰주소,주소(카카오),사업장명(카카오)
0,종로구,관철동,상해,분식,198867.702591,451824.281126,https://place.map.kakao.com/13333296#comment,서울 종로구 삼일대로 382 (관철동 5-12),상해
1,관악구,신림동,미쿠,일식,193687.348199,442284.164025,https://place.map.kakao.com/658611073#comment,서울 관악구 신림로59길 15-13 4층 (신림동 1638),미쿠
2,구로구,구로동,여름엔빙수겨울엔떡국,한식,189814.305919,443046.239859,https://place.map.kakao.com/1160052313#comment,서울 구로구 구로동로22길 21 (구로동 734-18),여름엔빙수 겨울엔떡국
3,강남구,대치동,카페디퍼,카페,204776.548264,444677.642985,https://place.map.kakao.com/637651089#comment,서울 강남구 삼성로85길 33 (대치동 894),카페디퍼
4,은평구,응암동,달의 작업실(Lalune),한식,192540.370555,454712.243026,https://place.map.kakao.com/1966713560#comment,서울 은평구 응암로21가길 9-11 (응암동 126-5),LALUNE달의작업실


### 4. 카카오맵에서 사라진 리뷰주소 제거 

In [7]:
error = len(df[df['사업장명(카카오)'] == 'error'])
print(f'리뷰주소 검색하면 "요청하신 곳에 해당하는 장소가 없어요" 라고 나오는 음식점{error}개') 
df[df['사업장명(카카오)'] == 'error'].head()

리뷰주소 검색하면 "요청하신 곳에 해당하는 장소가 없어요" 라고 나오는 음식점16개


,구,동,사업장명,업종,좌표정보(X),좌표정보(Y),리뷰주소,주소(카카오),사업장명(카카오)
3329,중구,흥인동,미인,카페,201392.574140,451476.315655,https://place.map.kakao.com/16063769#comment,error,error
12678,동대문구,장안동,니뽕내뽕바우하우스장안점,한식,206325.349432,452184.210161,https://place.map.kakao.com/2063731603#comment,error,error
14195,동대문구,장안동,히노아지,일식,206325.349432,452184.210161,https://place.map.kakao.com/608624108#comment,error,error
14851,동대문구,장안동,킹콩부대찌개장안아트몰링점,한식,206325.349432,452184.210161,https://place.map.kakao.com/1666308514#comment,error,error
25743,은평구,역촌동,황가네,카페,192508.832585,456133.279470,https://place.map.kakao.com/1154880397#comment,error,error


In [8]:
df = df[df['사업장명(카카오)'] != 'error']
print(df.shape)
df.head()

(91324, 9)


,구,동,사업장명,업종,좌표정보(X),좌표정보(Y),리뷰주소,주소(카카오),사업장명(카카오)
0,종로구,관철동,상해,분식,198867.702591,451824.281126,https://place.map.kakao.com/13333296#comment,서울 종로구 삼일대로 382 (관철동 5-12),상해
1,관악구,신림동,미쿠,일식,193687.348199,442284.164025,https://place.map.kakao.com/658611073#comment,서울 관악구 신림로59길 15-13 4층 (신림동 1638),미쿠
2,구로구,구로동,여름엔빙수겨울엔떡국,한식,189814.305919,443046.239859,https://place.map.kakao.com/1160052313#comment,서울 구로구 구로동로22길 21 (구로동 734-18),여름엔빙수 겨울엔떡국
3,강남구,대치동,카페디퍼,카페,204776.548264,444677.642985,https://place.map.kakao.com/637651089#comment,서울 강남구 삼성로85길 33 (대치동 894),카페디퍼
4,은평구,응암동,달의 작업실(Lalune),한식,192540.370555,454712.243026,https://place.map.kakao.com/1966713560#comment,서울 은평구 응암로21가길 9-11 (응암동 126-5),LALUNE달의작업실


### 5. 자치구 비교해서 맞지 않는 음식점 제거

In [9]:
add_list = df['주소(카카오)'].to_list() # 카카오주소에서 자치구 추출
gu_list = [add_list[i].split()[1] for i in range(len(add_list))]
df.insert(9,'구(카카오)',gu_list)
df.head()

,구,동,사업장명,업종,좌표정보(X),좌표정보(Y),리뷰주소,주소(카카오),사업장명(카카오),구(카카오)
0,종로구,관철동,상해,분식,198867.702591,451824.281126,https://place.map.kakao.com/13333296#comment,서울 종로구 삼일대로 382 (관철동 5-12),상해,종로구
1,관악구,신림동,미쿠,일식,193687.348199,442284.164025,https://place.map.kakao.com/658611073#comment,서울 관악구 신림로59길 15-13 4층 (신림동 1638),미쿠,관악구
2,구로구,구로동,여름엔빙수겨울엔떡국,한식,189814.305919,443046.239859,https://place.map.kakao.com/1160052313#comment,서울 구로구 구로동로22길 21 (구로동 734-18),여름엔빙수 겨울엔떡국,구로구
3,강남구,대치동,카페디퍼,카페,204776.548264,444677.642985,https://place.map.kakao.com/637651089#comment,서울 강남구 삼성로85길 33 (대치동 894),카페디퍼,강남구
4,은평구,응암동,달의 작업실(Lalune),한식,192540.370555,454712.243026,https://place.map.kakao.com/1966713560#comment,서울 은평구 응암로21가길 9-11 (응암동 126-5),LALUNE달의작업실,은평구


In [10]:
error = len(df[df['구'] != df['구(카카오)']])
print(f'자치구가 맞지 않는 음식점{error}개')

자치구가 맞지 않는 음식점1795개


In [11]:
df = df[df['구'] == df['구(카카오)']]
print(df.shape)
df.head()

(89529, 10)


,구,동,사업장명,업종,좌표정보(X),좌표정보(Y),리뷰주소,주소(카카오),사업장명(카카오),구(카카오)
0,종로구,관철동,상해,분식,198867.702591,451824.281126,https://place.map.kakao.com/13333296#comment,서울 종로구 삼일대로 382 (관철동 5-12),상해,종로구
1,관악구,신림동,미쿠,일식,193687.348199,442284.164025,https://place.map.kakao.com/658611073#comment,서울 관악구 신림로59길 15-13 4층 (신림동 1638),미쿠,관악구
2,구로구,구로동,여름엔빙수겨울엔떡국,한식,189814.305919,443046.239859,https://place.map.kakao.com/1160052313#comment,서울 구로구 구로동로22길 21 (구로동 734-18),여름엔빙수 겨울엔떡국,구로구
3,강남구,대치동,카페디퍼,카페,204776.548264,444677.642985,https://place.map.kakao.com/637651089#comment,서울 강남구 삼성로85길 33 (대치동 894),카페디퍼,강남구
4,은평구,응암동,달의 작업실(Lalune),한식,192540.370555,454712.243026,https://place.map.kakao.com/1966713560#comment,서울 은평구 응암로21가길 9-11 (응암동 126-5),LALUNE달의작업실,은평구
